# Constants

## Only if needed
```
import os

# Login with tenant ID
os.system("az login --tenant 3ad0b905-34ab-4116-93d9-c1dcc2d35af6 --output none")

# Set the subscription programmatically
os.system("az account set --subscription eca2eddb-0f0c-4351-a634-52751499eeea")
```

In [1]:
import os, json
from IPython.display import Markdown, display
from dotenv import load_dotenv # requires python-dotenv
from PIL import Image # requires pip install pillow
from datetime import datetime
from common.agents_helper_functions_NEW import *
import importlib.metadata
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

if not load_dotenv("./../config/credentials_my.env"):
    print("Environment variables not loaded, cell execution stopped")
else:
    print("Environment variables have been loaded ;-)")


project_endpoint = os.environ["PROJECT_ENDPOINT"]
deployment_name =  os.environ["MODEL_DEPLOYMENT_NAME"]
api_version = os.environ["OPENAI_API_VERSION"] # at least "2025-03-01-preview"

print(f'Project Endpoint: <{project_endpoint}>')
print(f"azure-ai-projects library installed version: {importlib.metadata.version("azure-ai-projects")}")
print(f"azure-ai-agents library installed version: {importlib.metadata.version("azure-ai-agents")}")

Environment variables have been loaded ;-)
Project Endpoint: <https://mmai-swc-new01-prj-resource.services.ai.azure.com/api/projects/mmai-swc-new01-prj>
azure-ai-projects library installed version: 1.0.0b11
azure-ai-agents library installed version: 1.1.0b1


# Create AI Foundry Project Client

In [2]:
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),
)

project_client

# Extract Azure OpenAI client from the AI Foundry Project

In [3]:
from openai import AzureOpenAI # <<<<< NOT NEEDED FOR AZURE AI FOUNDRY SDK

# using explicit type hinting for IntelliSense
azure_openai_client: AzureOpenAI = project_client.inference.get_azure_openai_client(
    api_version = api_version,
)

# if NOT MI --> azure_openai_client.api_key = os.environ.get("AZURE_OPENAI_API_KEY") 

print(azure_openai_client.base_url)

https://mmai-swc-new01-prj-resource.services.ai.azure.com/openai/


# Create OpenAI **Assistant** with CodeInterpreterTool, 
# from Azure OpenAI Client extracte from the AI Foundry Project

In [4]:
# First, create the code interpreter definitions...
code_interpreter_tool = {"type": "code_interpreter"}
code_interpreter_resources={"code_interpreter":{"file_ids":[]}}

# ...then, create the OpenAI Assistant Agent, attaching code interpreter
assistant = azure_openai_client.beta.assistants.create(
    name = "OpenAI Assistant #1",  
    model=deployment_name,
    instructions="You are a helpful agent",
    tools=[code_interpreter_tool],
    tool_resources=code_interpreter_resources,
    temperature=1,
    top_p=1
)

print(f"Assistant {assistant.name} ({assistant.id}) has been created. It's available in the AI Foundry Assistants playground.")

Assistant OpenAI Assistant #1 (asst_RywdmUzlGMF3Tod9wpmB2qx6) has been created. It's available in the AI Foundry Assistants playground.


# Create the thread and attach a new message to it

In [5]:
# Create a thread
thread = azure_openai_client.beta.threads.create()

print(f"The following thread has been created:\n{thread}\n")

# Add a user message to the thread
message = azure_openai_client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="""
    Could you please create a bar chart for the operating profit using 
    the following data and provide the file to me? 
    Company A: $1.2 million, Company B: $2.5 million, Company C: $3.0 million, 
    Company D: $1.8 million
    """,
)
print(f"The following message has been attached to thread <{thread.id}>:\n{message}")

# to retrieve the message of a thread, please run: azure_openai_client.beta.threads.messages.retrieve (thread_id=thread.id, message_id=message.id)

The following thread has been created:
Thread(id='thread_kwYpeQ7ovuAcZWSiloc5SQl8', created_at=1748501039, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

The following message has been attached to thread <thread_kwYpeQ7ovuAcZWSiloc5SQl8>:
Message(id='msg_DyjwbWcaRsNtNdISeDXaxgV9', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='\n    Could you please create a bar chart for the operating profit using \n    the following data and provide the file to me? \n    Company A: $1.2 million, Company B: $2.5 million, Company C: $3.0 million, \n    Company D: $1.8 million\n    '), type='text')], created_at=1748501039, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_kwYpeQ7ovuAcZWSiloc5SQl8')


# Run the agent syncrhonously

In [6]:
%%time

# Run the agent
run = azure_openai_client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)
print(f"Run status: {run.status}")

# Looping until the run completes or fails
import time
while run.status in ['queued', 'in_progress', 'cancelling']:
    time.sleep(1)
    run = azure_openai_client.beta.threads.runs.retrieve\
        (thread_id=thread.id, run_id=run.id)
    print(f"Run status: {run.status}")  

Run status: queued
Run status: in_progress
Run status: in_progress
Run status: in_progress
Run status: in_progress
Run status: in_progress
Run status: completed
CPU times: user 23.9 ms, sys: 4.2 ms, total: 28.1 ms
Wall time: 6.7 s


# Retrieve messages from the thread after the agent run execution

In [7]:
import openai

if run.status == 'completed':
    messages = azure_openai_client.beta.threads.messages.list(thread_id=thread.id)
    messages_nr = len(messages.data)
    print(f"Here are the {messages_nr} messages:\n")
    
    for i, message in enumerate(reversed(messages.data), 1):
        j =0
        print(f"\n===== MESSAGE {i} =====")
        for c in message.content:
            j += 1
            if (type(c) is openai.types.beta.threads.image_file_content_block.ImageFileContentBlock):
                print(f"\nCONTENT {j} (ImageFileContentBlock)--> image_file id: {c.image_file.file_id}")
            elif (type(c) is openai.types.beta.threads.text_content_block.TextContentBlock):
                print(f"\nCONTENT {j} (TextContentBlock) --> Text: {c.text.value}")
                for a in c.text.annotations:
                    print(f">>> Annotation in MessageTextContent {j} of message {i}: {a.text}\n")
else:
    print(f"Sorry, I can't proceed because the run status is {run.status}")

Here are the 2 messages:


===== MESSAGE 1 =====

CONTENT 1 (TextContentBlock) --> Text: 
    Could you please create a bar chart for the operating profit using 
    the following data and provide the file to me? 
    Company A: $1.2 million, Company B: $2.5 million, Company C: $3.0 million, 
    Company D: $1.8 million
    

===== MESSAGE 2 =====

CONTENT 1 (TextContentBlock) --> Text: The bar chart displaying the operating profit for the companies has been created. You can download the chart using the link below:

[Download the bar chart](sandbox:/mnt/data/operating_profit_bar_chart.png)
>>> Annotation in MessageTextContent 1 of message 2: sandbox:/mnt/data/operating_profit_bar_chart.png



# Retrieve and download eventual images
**Note**: sometimes, the image link is provided in the textual answer. In this case it won't be automatically shown

In [8]:
from IPython.display import Image, display
for c in json.loads(messages.to_json())["data"][0]["content"]:
    if c["type"]=="text":
        print(f'text: {c["text"]["value"]}')
    elif c["type"]=="image_file":
        file_id = c["image_file"]["file_id"]
        content = azure_openai_client.files.content(file_id=file_id)
        image= content.write_to_file(f"{file_id}.jpg")
        print(f'image_file: {file_id}')
        display(Image(filename=f"{file_id}.jpg"))

text: The bar chart displaying the operating profit for the companies has been created. You can download the chart using the link below:

[Download the bar chart](sandbox:/mnt/data/operating_profit_bar_chart.png)


# Teardown

In [9]:
# delete thread. Beaware: it's not possible to retrieve the threads list as it is with AI Foundry Agents!
azure_openai_client.beta.threads.delete(thread_id=thread.id)

ThreadDeleted(id='thread_kwYpeQ7ovuAcZWSiloc5SQl8', deleted=True, object='thread.deleted')

In [10]:
def list_assistants(aoc: AzureOpenAI, delete:bool=False):
    assistants = aoc.beta.assistants.list(limit=100).data

    i=0
    while len(assistants) > 0:
        for assistant in assistants:
            i=i+1
            if delete:
                print(f"Deleting assistant {i}: {assistant.id} ({assistant.name}) created at {datetime.fromtimestamp(assistant.created_at).strftime('%Y-%m-%d %H:%M:%S')}...")
                # client.beta.assistants.delete(assistant.id)
                aoc.beta.assistants.delete(assistant.id)
            else:
                print(f"Assistant {i}: {assistant.id} ({assistant.name}) created at {datetime.fromtimestamp(agent['created_at']).strftime('%Y-%m-%d %H:%M:%S')}...")

            assistants.remove(assistant)
            #agents = aoc.agents.list_agents(limit=100)['data']
    
    print (f"\n{i} assistants have been successfully processed.")


list_assistants(azure_openai_client, delete=True)

Deleting assistant 1: asst_RywdmUzlGMF3Tod9wpmB2qx6 (OpenAI Assistant #1) created at 2025-05-29 06:43:59...

1 assistants have been successfully processed.


# HIC SUNT LEONES